In [ ]:
'''
##### module #####

argparse
: 파이썬 파일을 실행시키면서 인자값을 새로 지정해주는 코드를 작성할 때 사용하는 모듈
: 기본적으로 터미널에서 실행하는 것을 원칙으로 하기 때문에,
  주피터 노트북 같은 대화형 실행 framework에서는 제대로 작동하지 않을 수도 있음
: parser.add_argument를 여러개 더 추가해서, 내가 받고 싶은 인자 개수만큼 만들어주면 됨
: 1. ArgumentParser에 원하는 description을 입력하여 parser 객체를 생성
  2. add_argument() method를 통해 원하는 만큼 인자 종류를 추가
  3. parse_args() method로 명령창에서 주어진 인자를 파싱
  4. args라는 이름으로 파싱을 성공했다면 args.parameter 형태로 주어진 인자 값을 받아 사용할 수 있음
: https://greeksharifa.github.io/references/2019/02/12/argparse-usage/
: https://brownbears.tistory.com/413

ex) 
parser.add_argument('--print-number', '-p', type=int, default=5, help='an integer for printing repeatably')

'--print-number', '-p'  
=> 인자 이름 지정할 때 여러 이름 가능(보통 1~2개)
 type=int               
 => 기본적으로 parse_args()가 주어진 인자들을 파싱할 때는 모든 문자를 숫자 등이 아닌 문자열 취급하기 때문에 데이터 타입을 지정해주어야 함
default=5               
=> 값을 저장할 때 명시적으로 지정하지 않았을 때 들어가는 기본값을 설정할 수 있음
help='~~'               
=>  description을 쓸 수 있음
  

glob
: 지정 경로의 파일을 가져오기 위한 모듈

json
: 

multiprocessing
: 프로세스 기반 병렬 처리

pathlib
: 파일 시스템 경로를 객체로 다룰 수 있음
: glob, os.path, shutil에서 사용했던 모든 기능을 pathlib만을 사용하여 구현할 수 있음
: https://wikidocs.net/110182

lr_scheduler
: 학습과정에서 learning rate를 조정할 수 있음
: LambdaLR, MultiplicativeLR, StepLR, MultiStepLR, ExponentialLR 등
  이 중 StepLR은 흔히 사용되는 learning rate scheduler 중 하나로써 step_size에 지정된 epoch 수 마다 이전 lr에 gamma만큼 곱해주는 방식임
: https://sanghyu.tistory.com/113
: https://easy-going-programming.tistory.com/10

tensorboard
: 신경망 학습 내역을 시각화하는 도구
: tensorboard를 사용하기 위해서는 SummaryWritter 인스턴스를 먼저 생성해야 함
: https://tutorials.pytorch.kr/recipes/recipes/tensorboard_with_pytorch.html


############################################################
https://hoya012.github.io/blog/reproducible_pytorch/
https://jh-bk.tistory.com/19
https://m.blog.naver.com/atelierjpro/221175394278

똑같은 세팅으로 실험을 돌렸음에도 불구하고 학습 loss나 세세한 inference 결과 등이 조금씩 차이나는 현상 발생 원인에 대하여 알아보자.

파이토치가 랜덤한 동작을 보이는 원인은 크게 아래 세가지로 나눌 수 있다.
- 파이토치 내/외부의 난수 생성기(random number generator  RNG)
- cuDNN의 컨볼루션 벤치마킹
- PyTorch에서 사용하는 알고리즘 자체의 non-determinism(비결정론적 알고리즘)

## Random number genetator(RNG)
   Python RNG
   : random.seed(특정 값)
   : 파이썬 내장 난수 생성 모듈이 사용될 가능성이 있으므로, 본격적인 코드 실행에 앞서 특정한 값으로 시드를 초기화 시켜주어야 함

   PyTorch RNG
   : torch.manual_seed(특정 값)
   : 파이썬 난수 생성기 이외에, 파이토치 자체의 난수 생성기도 시드를 설정해 주어야 함
   : torch.manual_seed() 를 호출하면 CPU와 모든 GPU 디바이스에 대해서 난수 시드가 설정됨

   Numpy RNG
   : np.random.seed(특정 값)
   : 만약 파이썬 내장 기능과 파이토치 이외에 다른 라이브러리 혹은 NumPy 라이브러리를 이용하고 있고, 
     해당 부분에 NumPy 난수 생성기가 관여하는 부분이 있다면 Numpy RNG에 대해서도 시드를 설정해야 함
   : 만약 위와 같이 기본 난수 생성기 이외에 random 모듈의 Generator 클래스 오브젝트를 따로 생성해 난수를 생성하고 있다면, 빼놓지 말고 따로 시드를 설정해 주자.
   
   Multi-process data loading
   : 추가로, 파이토치 데이터로더의 각 워커는 기본적으로 base_seed + worker_id 의 값으로 개별적으로 시드 세팅이 되기 때문에 worker_init_fn() 을 정의해 시드 세팅을 수동으로 해 주어야 함
   
   
## CUDA convolution benchmarking
   : torch.backends.cudnn.deterministic = True
     torch.backends.cudnn.benchmark = False    => Randomness 제어를 위한 설정법은 두 줄임
   : 이 벤치마킹 기능을 꺼둠으로써 convolution 알고리즘의 차이에 의한 랜덤성을 제거할 수 있다. 다만 이 경우에 최적의 알고리즘을 사용하지 않게 되므로, 성능 저하가 발생할 가능성도 있음
   : strict한 재현성이 필요 없는 상황에서는 웬만하면 benckmark 기능을 켜 두는 편이 성능 면에서 더 낫다고 알아두면 좋을 듯??
   
   
## Non-deterministic algorithm
   : torch.use_deterministic_algorithms(True)
   : 파이토치 내에는 기본적으로 non-deterministic 하게 작동하는 알고리즘들이 존재하며, 설정을 통해서 deterministic implementation으로 이들을 대체할 수 있음
   : 위에서 언급한 torch.backends.cudnn.benckmark 와는 별개로 따로 설정해 주어야 함
   : 이 설정이 적용되어 있으면, PyTorch의 모든 오퍼레이션들은 deterministic한 알고리즘으로 수행되며, 만약 deterministic implementation이 존재하지 않는다면 RuntimeError를 throw 해 줌
############################################################
 
 
random.choices()
: 지정 시퀀스 중 지정한 개수 만큼 가중치 반영해 랜덤 요소 반환. (시퀀스 : String, Range, List, Tuple, 기타 등등의 시퀀스)
: random.choices( 시퀀스(필수), 각 값에 대한 가중치(선택. 디폴트 None), 각 값에 대한 누적 가중치(선택. 디폴트 None), 반환 List의 요소 개수를 나타내는 정수(선택. 디폴트 1) )





'''

In [4]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
from importlib import import_module
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataset import MaskBaseDataset # dataset.py
from loss import create_criterion # loss.py


In [5]:

# 시드 설정 라인은 최대한 프로그램 앞 단에서 일괄적으로 실행시켜 혹시 모를 non-determinism을 방지하는 것이 좋다.
# 딥러닝 API들은 random number를 CPU에서 생성하기도 하고 GPU(cuda)에서 생성하기도 하기 때문에 둘 다 seed를 별도로 관리해야 한다.
def seed_everything(seed):
    torch.manual_seed(seed) # PyTorch RNG   # cpu 연산 무작위 고정 
    torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU  # 멀티 gpu 연산 무작위 고정 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) # Numpy RNG
    random.seed(seed) # Python RNG


In [6]:

def get_lr(optimizer):
    for param_group in optimizer.param_groups: # 각 그룹의 'lr'에 매핑된 값을 갱신
        return param_group['lr']


def grid_image(np_images, gts, preds, n=16, shuffle=False):
    batch_size = np_images.shape[0]
    assert n <= batch_size # [assert 조건] 이라 적었을 때 조건을 충족하지 않는다면 에러를 내라 할 때 사용

    choices = random.choices(range(batch_size), k=n) if shuffle else list(range(n))
    figure = plt.figure(figsize=(12, 18 + 2))  # cautions: hardcoded, 이미지 크기에 따라 figsize 를 조정해야 할 수 있습니다. T.T
    plt.subplots_adjust(top=0.8)               # cautions: hardcoded, 이미지 크기에 따라 top 를 조정해야 할 수 있습니다. T.T
    n_grid = np.ceil(n ** 0.5) # 올림. 입력값보다 같거나 큰 정수 중 가장 가까운 값을 반환함
    tasks = ["mask", "gender", "age"]
    for idx, choice in enumerate(choices):
        gt = gts[choice].item()
        pred = preds[choice].item()
        image = np_images[choice]
        # title = f"gt: {gt}, pred: {pred}"
        gt_decoded_labels = MaskBaseDataset.decode_multi_class(gt)
        pred_decoded_labels = MaskBaseDataset.decode_multi_class(pred)
        title = "\n".join([
            f"{task} - gt: {gt_label}, pred: {pred_label}"
            for gt_label, pred_label, task
            in zip(gt_decoded_labels, pred_decoded_labels, tasks)
        ])

        plt.subplot(n_grid, n_grid, idx + 1, title=title)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(image, cmap=plt.cm.binary)

    return figure


In [7]:

def increment_path(path, exist_ok=False):
    """ Automatically increment path, i.e. runs/exp --> runs/exp0, runs/exp1 etc.

    Args:
        path (str or pathlib.Path): f"{model_dir}/{args.name}".
        exist_ok (bool): whether increment path (increment if False). 경로 증가 여부(False인 경우 증가).
    """
    # path.exists()메소드는 주어진 경로가 존재하는지 확인하고 존재하면True를 반환하고 그렇지 않으면False를 반환함
    path = Path(path)
    if (path.exists() and exist_ok) or (not path.exists()): # (경로가 존재하지 않고 and 경로가 증가했으면) or (경로가 존재하지 않으면)
        return str(path)
    else:
        dirs = glob.glob(f"{path}*")
        matches = [re.search(rf"%s(\d+)" % path.stem, d) for d in dirs]
        i = [int(m.groups()[0]) for m in matches if m]
        n = max(i) + 1 if i else 2
        return f"{path}{n}"


In [8]:

def train(data_dir, model_dir, args):
    seed_everything(args.seed) # seed 고정

    save_dir = increment_path(os.path.join(model_dir, args.name)) # print해보기

    # -- settings
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # -- dataset
    # getattr(object, name[, default]): object에 존재하는 속성의 값을 가져옴
    dataset_module = getattr(import_module("dataset"), args.dataset)  # default: BaseAugmentation
    dataset = dataset_module(
        data_dir=data_dir,
    )
    num_classes = dataset.num_classes  # 18

    # -- augmentation
    transform_module = getattr(import_module("dataset"), args.augmentation)  # default: BaseAugmentation
    transform = transform_module(
        resize=args.resize,
        mean=dataset.mean,
        std=dataset.std,
    )
    dataset.set_transform(transform)

    # -- data_loader
    train_set, val_set = dataset.split_dataset() # class MaskBaseDataset(Dataset): -> def split_dataset(self) -> Tuple[Subset, Subset]: ?????

    train_loader = DataLoader(
        train_set,
        batch_size=args.batch_size,
        num_workers=multiprocessing.cpu_count()//2,
        shuffle=True,
        pin_memory=use_cuda, # dataloader의 리턴값을 gpu 를 쓰는 텐서로 변환
        drop_last=True,
    )

    val_loader = DataLoader(
        val_set,
        batch_size=args.valid_batch_size,
        num_workers=multiprocessing.cpu_count()//2,
        shuffle=False,
        pin_memory=use_cuda, 
        drop_last=True,
    )

    # -- model
    # https://algopoolja.tistory.com/56
    model_module = getattr(import_module("model"), args.model)  # default: BaseModel
    model = model_module(
        num_classes=num_classes
    ).to(device) # 모델을 GPU에 넣음
    model = torch.nn.DataParallel(model) # pytorch는 초기값으로 하나의 GPU 만 쓰도록 설정되어있지만
                                         #  DataParallel을 작동시킴으로써 multiple GPU 연산을 쉽게 실행할 수 있다.
    # -- loss & metric
    criterion = create_criterion(args.criterion)  # default: cross_entropy
    opt_module = getattr(import_module("torch.optim"), args.optimizer)  # default: SGD
    optimizer = opt_module(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=args.lr,
        weight_decay=5e-4
    )
    scheduler = StepLR(optimizer, args.lr_decay_step, gamma=0.5)

    # -- logging
    logger = SummaryWriter(log_dir=save_dir)
    with open(os.path.join(save_dir, 'config.json'), 'w', encoding='utf-8') as f:
        json.dump(vars(args), f, ensure_ascii=False, indent=4)

    best_val_acc = 0
    best_val_loss = np.inf
    for epoch in range(args.epochs):
        # train loop
        model.train()
        loss_value = 0
        matches = 0
        for idx, train_batch in enumerate(train_loader):
            inputs, labels = train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)

            loss.backward()
            optimizer.step()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()
            if (idx + 1) % args.log_interval == 0:
                train_loss = loss_value / args.log_interval
                train_acc = matches / args.batch_size / args.log_interval
                current_lr = get_lr(optimizer)
                print(
                    f"Epoch[{epoch}/{args.epochs}]({idx + 1}/{len(train_loader)}) || "
                    f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
                )
                logger.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
                logger.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)

                loss_value = 0
                matches = 0

        scheduler.step()

        # val loop
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            val_acc_items = []
            figure = None
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

                if figure is None:
                    inputs_np = torch.clone(inputs).detach().cpu().permute(0, 2, 3, 1).numpy()
                    inputs_np = dataset_module.denormalize_image(inputs_np, dataset.mean, dataset.std)
                    figure = grid_image(
                        inputs_np, labels, preds, n=16, shuffle=args.dataset != "MaskSplitByProfileDataset"
                    )

            val_loss = np.sum(val_loss_items) / len(val_loader)
            val_acc = np.sum(val_acc_items) / len(val_set)
            best_val_loss = min(best_val_loss, val_loss)
            if val_acc > best_val_acc:
                print(f"New best model for val accuracy : {val_acc:4.2%}! saving the best model..")
                torch.save(model.module.state_dict(), f"{save_dir}/best.pth")
                best_val_acc = val_acc
            torch.save(model.module.state_dict(), f"{save_dir}/last.pth")
            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )
            logger.add_scalar("Val/loss", val_loss, epoch)
            logger.add_scalar("Val/accuracy", val_acc, epoch)
            logger.add_figure("results", figure, epoch)
            print()



In [9]:

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    from dotenv import load_dotenv
    import os
    load_dotenv(verbose=True)

    # Data and model checkpoints directories
    parser.add_argument('--seed', type=int, default=42, help='random seed (default: 42)')
    parser.add_argument('--epochs', type=int, default=1, help='number of epochs to train (default: 1)')
    parser.add_argument('--dataset', type=str, default='MaskBaseDataset', help='dataset augmentation type (default: MaskBaseDataset)')
    parser.add_argument('--augmentation', type=str, default='BaseAugmentation', help='data augmentation type (default: BaseAugmentation)')
    parser.add_argument("--resize", nargs="+", type=list, default=[128, 96], help='resize size for image when training')
    parser.add_argument('--batch_size', type=int, default=64, help='input batch size for training (default: 64)')
    parser.add_argument('--valid_batch_size', type=int, default=1000, help='input batch size for validing (default: 1000)')
    parser.add_argument('--model', type=str, default='BaseModel', help='model type (default: BaseModel)')
    parser.add_argument('--optimizer', type=str, default='SGD', help='optimizer type (default: SGD)')
    parser.add_argument('--lr', type=float, default=1e-3, help='learning rate (default: 1e-3)')
    parser.add_argument('--val_ratio', type=float, default=0.2, help='ratio for validaton (default: 0.2)')
    parser.add_argument('--criterion', type=str, default='cross_entropy', help='criterion type (default: cross_entropy)')
    parser.add_argument('--lr_decay_step', type=int, default=20, help='learning rate scheduler deacy step (default: 20)')
    parser.add_argument('--log_interval', type=int, default=20, help='how many batches to wait before logging training status')
    parser.add_argument('--name', default='exp', help='model save at {SM_MODEL_DIR}/{name}')

    # Container environment
    parser.add_argument('--data_dir', type=str, default=os.environ.get('SM_CHANNEL_TRAIN', '/opt/ml/input/data/train/images'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR', './model'))

    args = parser.parse_args()
    print(args)

    data_dir = args.data_dir
    model_dir = args.model_dir

    train(data_dir, model_dir, args)

ModuleNotFoundError: No module named 'dotenv'